In [1]:
import re
import requests
from bs4 import BeautifulSoup

URL = "https://stripy.org/database"

page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [2]:
import pandas as pd

In [36]:
table = pd.read_html(URL)[0]

In [37]:
table

,Locus,Repeat type,Motif,Region,Path. repeats,Disease
0,AFF2,Standard,CCG,5â² UTR,&GreaterEqual;200,FRAXE mental retardation (FRAXE)
1,AR,Standard,CAG,Coding,&GreaterEqual;40,Spinal and bulbar muscular atrophy (SBMA)
2,ARX_1,Imperfect GCN,GCN,Coding,&GreaterEqual;23&GreaterEqual;18,Developmental and epileptic encephalopathy-1 (...
3,ARX_2,Imperfect GCN,GCN,Coding,&GreaterEqual;20&GreaterEqual;20&GreaterEqual;23,Developmental and epileptic encephalopathy-1 (...
4,ATXN10,Standard,ATTCT,Intron,&GreaterEqual;800,Spinocerebellar ataxia 10 (SCA10)
5,ATXN8OS,Standard,CTG,3â² UTR,&GreaterEqual;80,Spinocerebellar ataxia 8 (SCA8)
6,ATN1,Standard,CAG,Coding,&GreaterEqual;49,Dentatorubral-pallidoluysian atrophy (DRPLA)
7,ATXN1,Standard,CAG,Coding,&GreaterEqual;39,Spinocerebellar ataxia 1 (SCA1)
8,ATXN2,Standard,CAG,Coding,&GreaterEqual;35,Spinocerebellar ataxia 2 (SCA2)
9,ATXN3,Standard,CAG,Coding,&GreaterEqual;56,Spinocerebellar ataxia 3 (SCA3)


In [82]:
rows = []
for idx, i in table.iterrows():
    n_url = URL + '/' + i["Locus"]
    d = pd.read_html(n_url)
    d = d[0].set_index(0)
    ref_region = d.loc["Reference region"].iloc[0].strip().split(' ')[0]
    ref_region = re.split(":|-", ref_region)
    ref_region[1] = int(ref_region[1])
    ref_region[2] = int(ref_region[2])
    # Add repeat unit
    #ref_region.append(d.loc["Repeat unit"])
    
    rows.append(ref_region)
    
m_coords = pd.DataFrame(rows, columns=["chrom", "start", "end"])

In [84]:
table = table.join(m_coords)

In [86]:
stripy = table

In [89]:
patho = pd.read_csv("pathogenic_repeats.hg38.bed", sep='\t', header=None, names=["chrom", "start", "end", "meta"])

In [91]:
rows = []
for idx, i in patho.iterrows():
    x = i['meta'].split(';')
    Locus = x[0].split('=')[1]
    motifs = x[1].split('=')[1].split(',')
                       
    rows.append([Locus, motifs])
patho = patho.join(pd.DataFrame(rows, columns=["Locus", "Motifs"]))

In [93]:
patho.set_index("Locus", inplace=True, drop=False)

In [95]:
stripy.set_index("Locus", inplace=True, drop=False)

In [102]:
(~patho.index.isin(stripy.index)).sum()

10

In [104]:
(~stripy.index.isin(patho.index)).sum()

10

In [120]:
patho["source"] = "patho"
stripy["source"] = "stripy"
full = pd.concat([patho, stripy], axis=0)
full = full.sort_index()

In [122]:
full["source"].value_counts()

patho     56
stripy    56
Name: source, dtype: int64

In [125]:
full[["chrom", "start", "end", "source"]]

,chrom,start,end,source
Locus,,,,
AFF2,chrX,148500631,148500691,patho
AFF2,chrX,148500638,148500683,stripy
AFF3,chr2,100104799,100104824,patho
AR,chrX,67545316,67545385,patho
AR,chrX,67545316,67545385,stripy
...,...,...,...,...
YEATS2,chr3,183712187,183712222,stripy
ZIC2,chr13,99985448,99985494,patho
ZIC2,chr13,99985448,99985493,stripy


In [126]:
len(full["Locus"].unique())

66

In [128]:
x = full["Locus"].value_counts()
full.loc[]

,chrom,start,end,meta,Locus,Motifs,source,Repeat type,Motif,Region,Path. repeats,Disease
Locus,,,,,,,,,,,,
AFF2,chrX,148500631,148500691,ID=AFF2;MOTIFS=GCC;STRUC=(GCC)n,AFF2,[GCC],patho,NaN,NaN,NaN,NaN,NaN
AFF2,chrX,148500638,148500683,NaN,AFF2,NaN,stripy,Standard,CCG,5â² UTR,&GreaterEqual;200,FRAXE mental retardation (FRAXE)
AFF3,chr2,100104799,100104824,ID=AFF3;MOTIFS=GCC;STRUC=(GCC)n,AFF3,[GCC],patho,NaN,NaN,NaN,NaN,NaN
AR,chrX,67545316,67545385,ID=AR;MOTIFS=GCA;STRUC=(GCA)n,AR,[GCA],patho,NaN,NaN,NaN,NaN,NaN
AR,chrX,67545316,67545385,NaN,AR,NaN,stripy,Standard,CAG,Coding,&GreaterEqual;40,Spinal and bulbar muscular atrophy (SBMA)
...,...,...,...,...,...,...,...,...,...,...,...,...
YEATS2,chr3,183712187,183712222,NaN,YEATS2,NaN,stripy,Nested,TTTCA,Intron,&GreaterEqual;1000,Familial adult myoclonic epilepsy 4 (FAME4)
ZIC2,chr13,99985448,99985494,ID=ZIC2;MOTIFS=GCN;STRUC=(GCN)n,ZIC2,[GCN],patho,NaN,NaN,NaN,NaN,NaN
ZIC2,chr13,99985448,99985493,NaN,ZIC2,NaN,stripy,Imperfect GCN,GCN,Coding,&GreaterEqual;25,Holoprosencephaly 5 (HPE5)


In [132]:
x = full["Locus"].value_counts() 
full.loc[x[x != 2].index]

,chrom,start,end,meta,Locus,Motifs,source,Repeat type,Motif,Region,Path. repeats,Disease
TMEM185A,chrX,149631763,149631782,ID=TMEM185A;MOTIFS=GCC;STRUC=(GCC)n,TMEM185A,[GCC],patho,NaN,NaN,NaN,NaN,NaN
ZIC3,chrX,137566826,137566856,NaN,ZIC3,NaN,stripy,Standard,GCC,Coding,&GreaterEqual;12,X-linked VACTERL syndrome (VACTERLX)
TBX1,chr22,19766762,19766807,NaN,TBX1,NaN,stripy,Imperfect GCN,GCN,Coding,&GreaterEqual;25,Tetralogy of Fallot (TOF)
HOXA13_1,chr7,27199924,27199966,NaN,HOXA13_1,NaN,stripy,Imperfect GCN,GCN,Coding,&GreaterEqual;22,Hand-foot-genital syndrome (HFG)
PRDM12,chr9,130681606,130681639,NaN,PRDM12,NaN,stripy,Standard,GCC,Coding,&GreaterEqual;18,Hereditary sensory and autonomic neuropathy ty...
NOTCH2NL,chr1,149390802,149390841,ID=NOTCH2NL;MOTIFS=GGC;STRUC=(GGC)n,NOTCH2NL,[GGC],patho,NaN,NaN,NaN,NaN,NaN
HOXA13_3,chr7,27199678,27199732,NaN,HOXA13_3,NaN,stripy,Imperfect GCN,GCN,Coding,&GreaterEqual;24,Hand-foot-genital syndrome (HFG)
HOXA13_2,chr7,27199825,27199861,NaN,HOXA13_2,NaN,stripy,Imperfect GCN,GCN,Coding,&GreaterEqual;18,Hand-foot-genital syndrome (HFG)
AFF3,chr2,100104799,100104824,ID=AFF3;MOTIFS=GCC;STRUC=(GCC)n,AFF3,[GCC],patho,NaN,NaN,NaN,NaN,NaN
HOXA13,chr7,27199825,27199862,ID=HOXA13;MOTIFS=NGC;STRUC=(NGC)n,HOXA13,[NGC],patho,NaN,NaN,NaN,NaN,NaN


In [133]:
full.loc[x[x != 2].index]["source"].value_counts()

patho     10
stripy    10
Name: source, dtype: int64

In [134]:
len(patho), len(stripy)

(56, 56)

In [15]:
patho = pd.read_csv("Patho.tsv", sep='\t')

In [16]:
gnom = pd.read_csv("gnomad_roughtable.txt", sep='\t')

In [17]:
gnom.set_index("ID", inplace=True, drop=False)

In [18]:
patho.set_index("Locus", inplace=True, drop=False)

In [19]:
patho.index.isin(gnom.index).sum()

58

In [20]:
gnom.index.isin(patho.index).sum()

58

In [21]:
gnom[~gnom.index.isin(patho.index)]

,ID,Reference repeat unit,Region,Inheritance mode,Associated disease(s)
ID,,,,,
ATXN8OS,ATXN8OS,CTG,3'-UTR,Autosomal dominant,Spinocerebellar ataxia 8


In [22]:
len(patho)

66

In [24]:
patho.head()

,chrom,start,end,Locus,Motifs,source,Repeat type,Motif,Region,Path. repeats,Disease
Locus,,,,,,,,,,,
AFF2,chrX,148500631,148500691,AFF2,GCC,both,Standard,CCG,5'UTR,>=200,FRAXE mental retardation (FRAXE)
AFF3,chr2,100104799,100104824,AFF3,GCC,patho,NaN,NaN,NaN,NaN,NaN
AR,chrX,67545316,67545385,AR,GCA,both,Standard,CAG,Coding,>=40,Spinal and bulbar muscular atrophy (SBMA)
ARX_1,chrX,25013649,25013698,ARX_1,GCG,both,Imperfect GCN,GCN,Coding,>=23>=18,Developmental and epileptic encephalopathy-1 (...
ARX_2,chrX,25013529,25013565,ARX_2,NaN,stripy,Imperfect GCN,GCN,Coding,>=20>=20>=23,Developmental and epileptic encephalopathy-1 (...


In [25]:
gnom.head()

,ID,Reference repeat unit,Region,Inheritance mode,Associated disease(s)
ID,,,,,
AFF2,AFF2,GCC,5'-UTR,X-linked recessive,FRAXE mental retardation
AR,AR,GCA,coding: polyglutamine,X-linked recessive,Spinal and bulbar muscular atrophy
ARX_1,ARX_1,NGC,coding: polyalanine,X-linked recessive,"Developmental and epileptic encephalopathy-1, ..."
ARX_2,ARX_2,NGC,coding: polyalanine,X-linked recessive,"Developmental and epileptic encephalopathy-1, ..."
ATN1,ATN1,CAG,coding: polyglutamine,Autosomal dominant,Dentatorubral-pallidoluysian atrophy


In [27]:
gnom.loc[patho[patho['source'] == 'gnomad'].index]

,ID,Reference repeat unit,Region,Inheritance mode,Associated disease(s)
Locus,,,,,
PRNP,PRNP,CCTCATGGTGGTGGCTGGGGGCAG,coding: other,Autosomal dominant,Creutzfeldt-Jakob disease
VWA1,VWA1,GGCGCGGAGC,coding: other,Autosomal recessive,Hereditary motor neuropathy
EIF4A3,EIF4A3,CCTCGCTGTGCCGCTGCCGA,5'-UTR,Autosomal recessive,Richieri-Costa-Pereira syndrome
NIPA1,NIPA1,GCG,coding: polyalanine,Autosomal dominant,Amyotrophic lateral sclerosis


In [28]:
n_patho = patho.join(gnom["Inheritance mode"])

In [30]:
n_patho.to_csv("Patho_v2.txt", sep='\t', index=False)

In [4]:
data = pd.read_csv("Patho.tsv", sep='\t')
data[["Locus", "Disease"]]

,Locus,Disease
0,AFF2,FRAXE mental retardation (FRAXE)
1,AFF3,NaN
2,AR,Spinal and bulbar muscular atrophy (SBMA)
3,ARX_1,Developmental and epileptic encephalopathy-1 (...
4,ARX_2,Developmental and epileptic encephalopathy-1 (...
...,...,...
61,ZNF713,NaN
62,PRNP,Creutzfeldt-Jakob disease
63,VWA1,Hereditary motor neuropathy
64,EIF4A3,Richieri-Costa-Pereira syndrome
